In [1]:
!cat .env

STAGE=CCR_HOMOLOG


In [2]:
Company.objects.all()

<QuerySet [<Company: 385c3404-2c8b-4987-8015-9b93e044aa6d: CCR - AutoBAn>, <Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>, <Company: 161ab539-eea0-4c7c-8e9f-99058261f850: CCR - SPVias>, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>]>

In [16]:
import numpy as np
import random
from django.contrib.gis.geos import LineString, Point
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs
from helpers.strings import keys_to_snake_case, keys_to_camel_case,to_snake_case
from tqdm.notebook import tqdm
import random
import time
import json
from datetime import datetime

In [17]:
to_snake_case('dasdsaTheo')

'dasdsa_theo'

In [18]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [19]:
def get_connected_reference(company, resource, key):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return list(a['value'] for a in select_options)

In [20]:
lista=[
    "Monitoração de Terraplenos e Estruturas de Contenção",
]
len(lista)*20


20

In [21]:
company=Company.objects.get(name="CCR - Rio Sp")
occs=OccurrenceType.objects.filter(company__name="CCR - Rio Sp",name__in=lista).order_by('name')
roads=list(Road.objects.filter(company=company))
firm = list(Firm.objects.filter(company=company))
user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(name='Identificado',companies=company)

occs.count(),user,status,roads

(1,
 <User: rlcs - Kartado CS>,
 <ServiceOrderActionStatus: Identificado - REPORTING_STATUS>,
 [<Road: 2036: BR-101 SP>,
  <Road: 2037: BR-116 RJ>,
  <Road: 2038: BR-116 SP>,
  <Road: 2040: BR-101 RJ>,
  <Road: 2041: BR-101 RJ>,
  <Road: 1: BR-116 SP>,
  <Road: 2: BR-116 RJ>,
  <Road: 3: BR-116 RJ>,
  <Road: 16: BR-116 SP>])

In [22]:
start_date = "2023-01-01"
end_date = "2023-12-31"

start_timestamp = int(time.mktime(datetime.strptime(start_date, "%Y-%m-%d").timetuple()))
end_timestamp = int(time.mktime(datetime.strptime(end_date, "%Y-%m-%d").timetuple()))

In [23]:
int("{:.0f}".format(np.random.uniform(0, 10)))

3

In [24]:
objects=[]
for a in occs:
    #Quantidade de apontamentos
    for i in range(2000):
        
        #rodovia randomica + coordenadas randomicas
        ro=np.random.choice(roads)
        km=float(np.random.choice(list(r['km'] for i,r in ro.marks.items())))
        point, road = km_to_coordinates(ro, km)
        
        #Geração form_data randomico
        form_data={}
        for b in a.form_fields['fields']:
            if b['dataType'] == 'float':
                form_data[to_snake_case(b['apiName'])] = float("{:.2f}".format(np.random.uniform(0, 1)))
            if b['dataType'] == 'number':
                form_data[to_snake_case(b['apiName'])] = int("{:.0f}".format(np.random.uniform(0, 10)))
            if b['dataType'] == 'timestamp':
                #data randomica
                random_timestamp = random.randint(start_timestamp, end_timestamp)
                random_date = datetime.fromtimestamp(random_timestamp)
                form_data[to_snake_case(b['apiName'])] = random_date.strftime("%d-%m-%Y %H:%M")
            if b['dataType'] == 'select':
                form_data[to_snake_case(b['apiName'])] = np.random.choice([c['value'] for c in b['selectOptions']['options']])
            if b['dataType'] == 'selectMultiple':
                form_data[to_snake_case(b['apiName'])] = [np.random.choice([c['value'] for c in b['selectOptions']['options']])]
            if b['dataType'] == 'boolean':
                form_data[to_snake_case(b['apiName'])] = random.choice([True,False])
            if b['dataType'] == 'string' or b['dataType'] == 'textArea':
                form_data[to_snake_case(b['apiName'])] = b['displayName']
        objects.append(Reporting(
                company=company,
                firm=np.random.choice(firm),
                occurrence_type=a,
                lane=np.random.choice(get_connected_reference(company, 'reporting', 'lane')),
                road=road,
                road_name=road.name,
                direction=np.random.choice(get_connected_reference(company, 'reporting', 'direction')),
                created_by=user,
                status=status,
#                     found_at=datetime.datetime(2023, 5, 16, 0, 0).replace(tzinfo=utc),
                km=km,
                end_km_manually_specified=True,
                end_km=float(np.random.choice(list(r['km'] for i,r in road.marks.items()))),
                point=point,
                form_data=form_data
                ))
#     break

In [45]:
len(objects),objects[320].__dict__

(1160,
 {'_state': <django.db.models.base.ModelState at 0x7fbde0b4a450>,
  'uuid': UUID('955fd3b7-a3fd-464e-83d5-d267180e5e9a'),
  'number': '',
  'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
  'road_name': 'BR-116 RJ',
  'road_id': 2037,
  'km': 235.0,
  'end_km': 187.0,
  'km_reference': None,
  'project_km': 0,
  'project_end_km': None,
  'point': <Point object at 0x7fbde0b6ca30>,
  'direction': '9',
  'lane': '21',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'firm_id': UUID('89bfb142-a735-499b-bde5-05e9225199fc'),
  'occurrence_type_id': UUID('1220321a-69f7-42d5-bb13-ceb8212f4df2'),
  'form_data': {'ficha_barreira_nj': 'Ficha',
   'latitude_barreira_nj': 0.62,
   'longitude_barreira_nj': 0.37,
   'x_utm_barreira_nj': 0.36,
   'y_utm_barreira_nj': 0.99,
   'zona_utm_barreira_nj': 'Zona UTM',
   'dispositivo_entrada': False,
   'dispositivo_pontual': '1',
   'dispositivo_pontual_engastado': '1',

In [25]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/2000 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.00053
Searching KM on direction...
Searching KM without direction...
Found KM on Road 16: BR-116 SP
CCR-TER-2023.00054
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.00055
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-TER-2023.00056
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.00057
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.00058
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.00059
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-TER-2023.00060
Searching KM on direction...
Searching KM without direction...
Found KM on Road 16: BR-116 SP
CCR-TER-2023.00061
Searching KM on

Found KM on Road 2041: BR-101 RJ
CCR-TER-2023.00127
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.00128
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.00129
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.00130
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.00131
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.00132
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.00133
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.00134
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.00135
Searching KM on direction...
Searchin

CCR-TER-2023.00201
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-TER-2023.00202
Searching KM on direction...
Searching KM without direction...
Found KM on Road 16: BR-116 SP
CCR-TER-2023.00203
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.00204
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.00205
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-TER-2023.00206
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-TER-2023.00207
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.00208
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.00209
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
CCR-TER-20

CCR-TER-2023.00275
Searching KM on direction...
Searching KM without direction...
Found KM on Road 16: BR-116 SP
CCR-TER-2023.00276
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.00277
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.00278
Searching KM on direction...
Found KM on Road 16: BR-116 SP
CCR-TER-2023.00279
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.00280
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.00281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.00282
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.00283
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2

...

Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.01752
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.01753
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.01754
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01755
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.01756
Searching KM on direction...
Searching KM without direction...
Found KM on Road 16: BR-116 SP
CCR-TER-2023.01757
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01758
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
CCR-TER-2023.01759
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01760
Se

CCR-TER-2023.01825
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01826
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.01827
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01828
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.01829
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.01830
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01831
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.01832
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.01833
Searching KM on direction...
Searching KM without direction...
F

CCR-TER-2023.01898
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.01899
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-TER-2023.01900
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
CCR-TER-2023.01901
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01902
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.01903
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01904
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.01905
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.01906
Searching KM on direction...
Searching KM without direction...
Foun

CCR-TER-2023.01972
Searching KM on direction...
Searching KM without direction...
Found KM on Road 16: BR-116 SP
CCR-TER-2023.01973
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
CCR-TER-2023.01974
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.01975
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
CCR-TER-2023.01976
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.01977
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.01978
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-TER-2023.01979
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-TER-2023.01980
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-

Searching KM without direction...
Found KM on Road 2038: BR-116 SP
CCR-TER-2023.02047
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
CCR-TER-2023.02048
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
CCR-TER-2023.02049
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-TER-2023.02050
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.02051
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
CCR-TER-2023.02052


In [8]:
random_timestamp = random.randint(start_timestamp, end_timestamp)
random_date = datetime.fromtimestamp(random_timestamp)

json_data = json.dumps({"date": random_date.strftime("%Y-%m-%d %H:%M:%S")})
print(json_data)

{"date": "2023-02-12 22:13:19"}


  0%|          | 0/80 [00:00<?, ?it/s]

041fd8cc-1980-4f49-b4d7-1a0e7329a16a
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
0d654c80-1fa4-4f29-a685-c4135e048f2d
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
8feb4bee-8674-4bee-8ffe-eef38659becc
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
12bdc832-ee42-45f6-b186-b2e9f61a8721
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
c49dd1fe-8f33-4715-bdb9-6a88d085ab5d
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
445a21ee-7a13-49c8-9fed-6802c4a2d1ef
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
94e81b83-0eb4-449f-8f65-32e07d9ae437
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
a8c0cf08-d4be-4698-a271-26045c9874ef
Searching KM on direction...
Sea

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
320bafde-63fb-4f04-b98d-2827a1b2a857
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
d8716dc2-a6b5-496c-909a-60715a3f0606
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
7c468793-e120-4b77-9d82-2b00f5cae98d
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
5a229617-78a7-48ab-8270-08b10ff5d8ab
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
3a698e83-6daa-41eb-bda6-f1e2329e6442
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
d2d3a1d8-82ba-4e06-b533-65bb45ca181a
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
1301aaa7-1727-4bb8-b61d-4d551f76b26a
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
d71cd562-6

In [7]:
a.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fbdc15d9790>,
 'uuid': UUID('0de2ccfa-92ad-4337-b65f-ad0974b6b87c'),
 'number': 'CCR-DEF-2023.01316',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-116 RJ',
 'road_id': 2,
 'km': 270.0,
 'end_km': 325.0,
 'km_reference': None,
 'project_km': 0.0,
 'project_end_km': 0.0,
 'point': <Point object at 0x7fbdc15d8b50>,
 'direction': '4',
 'lane': '1',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('c51e2071-67ca-44ff-ad4a-4a005cf95591'),
 'occurrence_type_id': UUID('0e28ebc6-bf8c-453c-875b-719c3261351c'),
 'form_data': {'year': 0.99,
  'color': '1',
  'index': 3,
  'notes': 'Observações',
  'other': 'Outra interferência',
  'semester': '1',
  'condition': '2',
  'fifth_lca': 0.13,
  'fifth_zpa': 0.85,
  'first_lca': 0.53,
  'first_zpa': 0.26,
  'ninth_lca': 0.41,
  'ninth_zpa': 0.03,
  'sixth_lca': 0.66,
  'sixth_zpa': 0.21,
  

In [12]:
for a in Reporting.objects.filter(occurrence_type__name='Monitoração de Sinalização Vertical',company=company):
    form_data={}
    for b in a.occurrence_type.form_fields['fields']:
            if b['dataType'] == 'float':
                form_data[to_snake_case(b['apiName'])] = float("{:.2f}".format(np.random.uniform(0, 1)))
            if b['dataType'] == 'number':
                form_data[to_snake_case(b['apiName'])] = int("{:.0f}".format(np.random.uniform(0, 10)))
            if b['dataType'] == 'timestamp':
                #data randomica
                random_timestamp = random.randint(start_timestamp, end_timestamp)
                random_date = datetime.fromtimestamp(random_timestamp)
                form_data[to_snake_case(b['apiName'])] = random_date.strftime("%d-%m-%Y %H:%M")
            if b['dataType'] == 'select':
                form_data[to_snake_case(b['apiName'])] = np.random.choice([c['value'] for c in b['selectOptions']['options']])
            if b['dataType'] == 'selectMultiple':
                form_data[to_snake_case(b['apiName'])] = [np.random.choice([c['value'] for c in b['selectOptions']['options']])]
            if b['dataType'] == 'boolean':
                form_data[to_snake_case(b['apiName'])] = random.choice([True,False])
            if b['dataType'] == 'string' or b['dataType'] == 'textArea':
                form_data[to_snake_case(b['apiName'])] = b['displayName']
    a.form_data=form_data
    a.save()

Searching KM on direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM with